In [38]:
!pip install transformers datasets torch scikit-learn pandas

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
!ls "/content/drive/MyDrive/subtask_A"


subtaskA.ipynb			task_a_trial.parquet
task_a_test_set_sample.parquet	task_a_validation_set.parquet
task_a_training_set_1.parquet


In [41]:
import pandas as pd
train_df = pd.read_parquet("/content/drive/MyDrive/subtask_A/task_a_training_set_1.parquet")
train_df.head()


,code,generator,label,language
0,"(a, b, c, d) = [int(x) for x in input().split(...",human,0,Python
1,valid version for the language; all others can...,Qwen/Qwen2.5-Coder-1.5B,1,Python
2,python\ndef min_cards_to_flip(s):\n vowels ...,Qwen/Qwen2.5-Coder-7B-Instruct,1,Python
3,T = int(input())\nfor t in range(T):\n\tcolor ...,human,0,Python
4,for i in range(int(input())):\n\tinput()\n\ta ...,human,0,Python


In [42]:
valid_df = pd.read_parquet("/content/drive/MyDrive/subtask_A/task_a_validation_set.parquet")
valid_df.head()

,code,generator,label,language
0,import sys\ninf = float('inf')\nfrom bisect im...,human,0,Python
1,"def Range(a): return min(max(a, -10), 10)\n\nd...",meta-llama/Llama-3.1-8B,1,Python
2,#include<bits/stdc++.h>\n\n\n\nusing namespace...,human,0,C++
3,t = int(input())\nfor _ in range(t):\n\tn = in...,human,0,Python
4,"def subArraySum(arr, n, target_sum):\n coun...",Qwen/Qwen2.5-Coder-32B-Instruct,1,Python


In [43]:
test_df  = pd.read_parquet("/content/drive/MyDrive/subtask_A/task_a_test_set_sample.parquet")
test_df.head()

,code,generator,label,language
0,public Vector To(Vector o)\n {\n ...,Human,0,C#
1,func (v *DefaultMessageSyntaxValidator) Valida...,Human,0,Go
2,"""""""Module managing testsuite capabilities\n\nC...",Human,0,Python
3,void Anvil::Image::on_memory_backing_opaque_up...,Human,0,C++
4,bool NOMAD::Priority_Eval_Point::dominates\n( ...,Human,0,C++


In [44]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds  = Dataset.from_pandas(test_df)

train_ds, valid_ds, test_ds


(Dataset({
     features: ['code', 'generator', 'label', 'language'],
     num_rows: 500000
 }),
 Dataset({
     features: ['code', 'generator', 'label', 'language'],
     num_rows: 100000
 }),
 Dataset({
     features: ['code', 'generator', 'label', 'language'],
     num_rows: 1000
 }))

In [45]:
from transformers import AutoTokenizer
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [46]:
def tokenize_function(example):
    return tokenizer(
        example["code"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

train_ds = train_ds.map(tokenize_function, batched=True)
valid_ds = valid_ds.map(tokenize_function, batched=True)
test_ds  = test_ds.map(tokenize_function, batched=True)


Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [47]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base",
    num_labels=2   # 0 = human, 1 = machine
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
import os

os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
os.environ["HF_HUB_OFFLINE"] = "1"


In [65]:
from huggingface_hub import HfFolder
HfFolder.delete_token()

In [66]:
!huggingface-cli logout

⚠️  Warning: 'huggingface-cli logout' is deprecated. Use 'hf auth logout' instead.
Not logged in!


In [67]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results_taskA",
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    push_to_hub=False
)


In [59]:
!pip install evaluate

In [60]:
import evaluate

f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {"macro_f1": f1_metric.compute(predictions=preds, references=labels, average="macro")}


In [61]:
from huggingface_hub import login
login(token=None)

In [68]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3229733835.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [69]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:


Abort: 